In [7]:
import pdb
import numpy as np
import gcp.bigquery as bq
import gcp.storage as storage
from sklearn.pipeline import Pipeline
try:
   import cPickle as pickle
except:
   import pickle
EST_PICKLE_FILENAME = 'baseline_final_estimator.pkl'

# First feature HAS to be 'district_id' for MAPE calculation.
fields_str = """
district_id	timeofday_slot	day_in_week	is_sunday	sum_price	avg_price	poi1	poi2	poi3
	poi4	poi5	traffic_tj_level1	traffic_tj_level2	traffic_tj_level3	traffic_tj_level4
	weather	weather_pm25	weather_temperature	gap
"""
fields = map(lambda x: x.strip(), fields_str.split('\t'))
features = fields[1:]

# Scorer Creation (MAPE)

In [8]:
def mape(X, predictions, y):
  num_timeslots = 43
  num_districts = 66
  if len(y.shape) == 1:
    y = np.asmatrix(y)
  if len(predictions.shape) == 1:
    predictions = np.asmatrix(predictions)
  Xy = np.concatenate((X, y.T, predictions.T), axis=1)
  districts = np.unique(X[:,0])
  district_scores = np.zeros(len(districts))
  for counter, key in enumerate(districts):
    group = np.compress((Xy[:,0] == key).flat, Xy, axis=0)
    district_scores[counter] = np.sum(np.absolute(
        (group[:,-2] -
         group[:,-1])/
        group[:,-2]
      )) / num_timeslots
  return np.sum(district_scores) / num_districts

def mape_scorer(estimator, X, y):
  predictions = estimator.predict(X)
  return -mape(X, predictions, y)

Testing MAPE

In [9]:
from sklearn.linear_model import LogisticRegression

est = LogisticRegression()
X = np.array([[1, 1], [1, 2], [2, 3], [2, 4]])
predictions = np.array([1, 2, 3, 4])
y = np.array([1, 2, 3, 4])

# Should return 0.0
mape(X, predictions, y)

0.0

# Feature Selection

In [10]:
%%sql --module q_all

SELECT *, HASH(CAST(district_id AS STRING) +timeslot) AS hash_value,
  IF(ABS(HASH(CAST(district_id AS STRING) + timeslot)) % 2 == 1, 'True', 'False')
    AS included_in_sample
FROM [datalab-projects-1331:xjk_algo_comp.future_gaps_final1]
WHERE gap > 0

# The above query randomizes its outputs.

In [11]:
query = bq.Query(q_all)
tableresult = query.results()

In [12]:
all_data = np.zeros((tableresult.length, len(fields)))
print 'there are {} rows'.format(tableresult.length)
for rcounter, row in enumerate(tableresult):
  for fcounter, field in enumerate(fields):
    all_data[rcounter, fcounter] = row[field]
  if rcounter % 5000 == 0:
    print 'processed {} rows'.format(rcounter)

there are 102680 rows
there are 102680 rows
there are 102680 rows
processed 0 rows
processed 0 rows
processed 0 rows
processed 5000 rows
processed 5000 rows
processed 5000 rows
processed 10000 rows
processed 10000 rows
processed 10000 rows
processed 15000 rows
processed 15000 rows
processed 15000 rows
processed 20000 rows
processed 20000 rows
processed 20000 rows
processed 25000 rows
processed 25000 rows
processed 25000 rows
processed 30000 rows
processed 30000 rows
processed 30000 rows
processed 35000 rows
processed 35000 rows
processed 35000 rows
processed 40000 rows
processed 40000 rows
processed 40000 rows
processed 45000 rows
processed 45000 rows
processed 45000 rows
processed 50000 rows
processed 50000 rows
processed 50000 rows
processed 55000 rows
processed 55000 rows
processed 55000 rows
processed 60000 rows
processed 60000 rows
processed 60000 rows
processed 65000 rows
processed 65000 rows
processed 65000 rows
processed 70000 rows
processed 70000 rows
processed 70000 rows
proc

In [13]:
# Get timeslots to test from GCS
item = storage.Item('datalab-projects-1331-datalab','data/timeslots_to_test.txt')
timeslots_to_test = item.read_from().strip().split('\n')
tquery = ','.join(map(lambda x: "'{}'".format(x), timeslots_to_test))
print(tquery)

'2016-01-22-46','2016-01-22-58','2016-01-22-70','2016-01-22-82','2016-01-22-94','2016-01-22-106','2016-01-22-118','2016-01-22-130','2016-01-22-142','2016-01-24-58','2016-01-24-70','2016-01-24-82','2016-01-24-94','2016-01-24-106','2016-01-24-118','2016-01-24-130','2016-01-24-142','2016-01-26-46','2016-01-26-58','2016-01-26-70','2016-01-26-82','2016-01-26-94','2016-01-26-106','2016-01-26-118','2016-01-26-130','2016-01-26-142','2016-01-28-58','2016-01-28-70','2016-01-28-82','2016-01-28-94','2016-01-28-106','2016-01-28-118','2016-01-28-130','2016-01-28-142','2016-01-30-46','2016-01-30-58','2016-01-30-70','2016-01-30-82','2016-01-30-94','2016-01-30-106','2016-01-30-118','2016-01-30-130','2016-01-30-142'


In [14]:
%%sql --module q_all_t

SELECT *
FROM [datalab-projects-1331:xjk_algo_comp_test.future_gaps_final1]
WHERE gap > 0 AND timeslot NOT IN ('2016-01-22-46','2016-01-22-58','2016-01-22-70','2016-01-22-82',
    '2016-01-22-94','2016-01-22-106','2016-01-22-118','2016-01-22-130','2016-01-22-142',
    '2016-01-24-58','2016-01-24-70','2016-01-24-82','2016-01-24-94','2016-01-24-106',
    '2016-01-24-118','2016-01-24-130','2016-01-24-142','2016-01-26-46','2016-01-26-58',
    '2016-01-26-70','2016-01-26-82','2016-01-26-94','2016-01-26-106','2016-01-26-118',
    '2016-01-26-130','2016-01-26-142','2016-01-28-58','2016-01-28-70','2016-01-28-82',
    '2016-01-28-94','2016-01-28-106','2016-01-28-118','2016-01-28-130','2016-01-28-142',
    '2016-01-30-46','2016-01-30-58','2016-01-30-70','2016-01-30-82','2016-01-30-94',
    '2016-01-30-106','2016-01-30-118','2016-01-30-130','2016-01-30-142')
ORDER BY timeslot, district_id

# Test dataset - used to check if estimator can generalize well to new data.

In [15]:
query_t = bq.Query(q_all_t)
tableresult_t = query_t.results()

In [16]:
all_data_t = np.zeros((tableresult_t.length, len(fields)))
print 'there are {} rows'.format(tableresult_t.length)
for rcounter, row in enumerate(tableresult_t):
  for fcounter, field in enumerate(fields):
    all_data_t[rcounter, fcounter] = row[field]
  if rcounter % 1000 == 0:
    print 'processed {} rows'.format(rcounter)

there are 3509 rows
there are 3509 rows
there are 3509 rows
processed 0 rows
processed 1000 rows
processed 0 rows
processed 1000 rows
processed 0 rows
processed 1000 rows
processed 2000 rows
processed 2000 rows
processed 2000 rows
processed 3000 rows
processed 3000 rows
processed 3000 rows


# Building and Testing Algorithm(s)

In [17]:
# Useful code to check NaN and Inf values. This is needed since these values would
# cause "Input contains NaN, infinity or a value too large for dtype('float32')
# errors when left unchecked.
print "Checkinf for NaN and Inf"
print "np.nan=", np.where(np.isnan(all_data))
print "is.inf=", np.where(np.isinf(all_data))
print "np.max=", np.max(abs(all_data))

Checkinf for NaN and Inf
np.nan= (array([     2,      2,      2, ..., 102679, 102679, 102679]), array([15, 16, 17, ..., 15, 16, 17]))
is.inf= (array([], dtype=int64), array([], dtype=int64))
np.max= nan
Checkinf for NaN and Inf
np.nan= (array([     2,      2,      2, ..., 102679, 102679, 102679]), array([15, 16, 17, ..., 15, 16, 17]))
is.inf= (array([], dtype=int64), array([], dtype=int64))
np.max= nan


In [18]:
all_data[np.isnan(all_data)] = 0
all_data_t[np.isnan(all_data_t)] = 0

In [19]:
# Useful code to check NaN and Inf values. This is needed since these values would
# cause "Input contains NaN, infinity or a value too large for dtype('float32')
# errors when left unchecked.
print "Checkinf for NaN and Inf"
print "np.nan=", np.where(np.isnan(all_data))
print "is.inf=", np.where(np.isinf(all_data))
print "np.max=", np.max(abs(all_data))

Checkinf for NaN and Inf
np.nan= (array([], dtype=int64), array([], dtype=int64))
is.inf= (array([], dtype=int64), array([], dtype=int64))
np.max= 708222.866131
Checkinf for NaN and Inf
np.nan= (array([], dtype=int64), array([], dtype=int64))
is.inf= (array([], dtype=int64), array([], dtype=int64))
np.max= 708222.866131
Checkinf for NaN and Inf
np.nan= (array([], dtype=int64), array([], dtype=int64))
is.inf= (array([], dtype=int64), array([], dtype=int64))
np.max= 708222.866131


In [31]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
# from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.preprocessing import Imputer
# from sklearn.grid_search import RandomizedSearchCV
from sklearn.grid_search import GridSearchCV


steps = [
#   ('impute', Imputer(0)),
#   ('feature_selection', SelectKBest(f_classif)),
  ('estimate', AdaBoostRegressor())
]

est = Pipeline(steps)

data_train = all_data[:,1:]
targets_train = all_data[:,0]
data_test = all_data_t[:,1:]
targets_test = all_data_t[:,0]

params = {
#   "feature_selection__k": [i for i in range(1, len(features) - 1)]
#   'estimate__max_features': [i for i in range(1, len(features))],
#   'estimate__n_estimators': [5, 10, 15, 20, 30]
  'estimate_learning_rate': [0.1, 0.3, 0.7, 1, 3, 5, 10],
  'estimate__n_estimators': [5, 10, 20, 50, 60, 80]
}
# cross_validation_iter = StratifiedShuffleSplit(y=targets_train, test_size=0.3,
#                                                random_state=RANDOM_STATE, n_iter=10)
# search_params = RandomizedSearchCV(
#   estimator=est,
#   param_distributions=params,
# #   cv=10,
#   scoring=mape_scorer,
#   n_jobs=2,
#   n_iter=5
# )

search_params = GridSearchCV(
  estimator=est,
  param_grid=params,
  cv=5,
  scoring=mape_scorer,
  n_jobs=2,
  verbose=1
)

search_params.fit(data_train, targets_train)
print(search_params.grid_scores_)
print(search_params.best_params_)
print(search_params.best_score_)
search_params.best_estimator_

Fitting 5 folds for each of 42 candidates, totalling 210 fits
Fitting 5 folds for each of 42 candidates, totalling 210 fits
Fitting 5 folds for each of 42 candidates, totalling 210 fits
An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line statement', (2, 0))

An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line statement', (2, 0))

An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line statement', (2, 0))

An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line statement', (2, 0))

An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line 

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
    ...........................................................................
/usr/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    157     pkg_name = mod_name.rpartition('.')[0]
    158     main_globals = sys.modules["__main__"].__dict__
    159     if alter_argv:
    160         sys.argv[0] = fname
    161     return _run_code(code, main_globals, None,
--> 162                      "__main__", fname, loader, pkg_name)
        fname = '/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = 'ipykernel'
    163 
    164 def run_module(mod_name, init_globals=None,
    165                run_name=None, alter_sys=False):
    166     """Execute a module's code without importing it

.........................................................

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
    ...........................................................................
/usr/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    157     pkg_name = mod_name.rpartition('.')[0]
    158     main_globals = sys.modules["__main__"].__dict__
    159     if alter_argv:
    160         sys.argv[0] = fname
    161     return _run_code(code, main_globals, None,
--> 162                      "__main__", fname, loader, pkg_name)
        fname = '/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = 'ipykernel'
    163 
    164 def run_module(mod_name, init_globals=None,
    165                run_name=None, alter_sys=False):
    166     """Execute a module's code without importing it

.........................................................

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
    ...........................................................................
/usr/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    157     pkg_name = mod_name.rpartition('.')[0]
    158     main_globals = sys.modules["__main__"].__dict__
    159     if alter_argv:
    160         sys.argv[0] = fname
    161     return _run_code(code, main_globals, None,
--> 162                      "__main__", fname, loader, pkg_name)
        fname = '/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = 'ipykernel'
    163 
    164 def run_module(mod_name, init_globals=None,
    165                run_name=None, alter_sys=False):
    166     """Execute a module's code without importing it

.........................................................

Test data's prediction MAPE score:

In [28]:
final_est = search_params.best_estimator_
test_predictions = final_est.predict(data_test)
print(mape(data_test, test_predictions, targets_test))

2.34569095992
2.34569095992
2.34569095992


In [29]:
pickle.dump(final_est, open(EST_PICKLE_FILENAME, "w") )

Run "Process Final Test Data With Final Algorithm" to use pickled final algorithm against final test data to produce csv required by this competition.

In [23]:
# Just testing Imputer. Turns out somehow Imputer causes number of features reduced, weird.

# imputer = Imputer()
est = DecisionTreeRegressor(max_features=len(features))

data_train_i = np.copy(data_train)
print(data_train.shape)
print(data_train[0:10])
# data_train_i = imputer.fit_transform(data_train)
data_train_i[np.isnan(data_train_i)] = 0
data_train_i.astype('float32')
print(data_train_i.shape)
print(data_train_i[0:10])
est.fit(data_train_i, targets_train)
predictions = est.predict(data_test)
print(mape(data_test, predictions, targets_test))

(102680, 18)
[[  1.70000000e+01   3.00000000e+00   0.00000000e+00   3.27000000e+02
    2.97272727e+01   1.87671617e+04  -2.45115880e+03  -1.62980717e+04
   -2.21621266e+04  -1.19145310e+03   7.28000000e+02   7.40000000e+01
    2.30000000e+01   2.90000000e+01   2.00000000e+00   1.06000000e+02
    1.00000000e+00   3.00000000e+00]
 [  1.09000000e+02   4.00000000e+00   0.00000000e+00   1.27000000e+02
    1.81428571e+01  -6.11945684e+04   1.66019762e+03   2.40659554e+02
    7.81203839e+03   3.14309459e+02   1.23000000e+02   1.90000000e+01
    3.00000000e+00   1.00000000e+00   3.00000000e+00   1.24000000e+02
    8.00000000e+00   4.00000000e+00]
 [  9.70000000e+01   4.00000000e+00   0.00000000e+00   2.49000000e+02
    1.55625000e+01  -5.73741656e+04  -3.33111579e+03   2.33390366e+03
    3.77453856e+03  -7.61532298e+02   2.34000000e+02   6.60000000e+01
    1.80000000e+01   8.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   2.00000000e+00]
 [  3.50000000e+01   6.00000000e+00 